We want to check if the solution imprelementation is correct

In [13]:
import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import interact, FloatSlider

import sys
from importlib import reload
sys.path.append("../scripts")
import solution as sol
sol = reload(sol)

In [14]:
bins = 500


def plot_dist(m0,r):    
    plt.figure(figsize=(7,4))
    plt.ylim(0, 2)
    plt.scatter(np.linspace(-1,1,bins), sol.sol(1000,r,m0,bins), color = "r")
    plt.title(f"Distribution")
    plt.legend()
    plt.grid(True)
    plt.show()

interact(
    plot_dist,
    m0 = FloatSlider(value=0.0, min=-1, max=1, step=0.01, description='m0'),
    r = FloatSlider(value=0.0, min=0, max=0.1, step=0.001, description='r')
)

interactive(children=(FloatSlider(value=0.0, description='m0', max=1.0, min=-1.0, step=0.01), FloatSlider(valu…

<function __main__.plot_dist(m0, r)>

In [16]:
def plot_pi(m0):
    r = np.linspace(0, 0.1, 100)
    pi0 = np.array([sol.pi0(1000,i, m0) for i in r])
    pi1 = np.array([sol.pi1(1000,i, m0) for i in r])

    plt.figure(figsize=(7,4))
    plt.ylim(0, 1.1)
    plt.plot(r, pi0, label='π₀')
    plt.plot(r, pi1, label='π₁')
    plt.plot(r, pi1+pi0, label='π₁')
    plt.xlabel("r")
    plt.ylabel("π")
    plt.title(f"π₀(r) and π₁(r)")
    plt.legend()
    plt.grid(True)
    plt.show()

interact(
    plot_pi,
    m0 = FloatSlider(value=0.0, min=-1, max=1, step=0.01, description='m0')
)

interactive(children=(FloatSlider(value=0.0, description='m0', max=1.0, min=-1.0, step=0.01), Output()), _dom_…

<function __main__.plot_pi(m0)>

0.0